# Infer NUCLEI - part 1

--------------

## OBJECTIVE:  Infer sub-cellular component #1: NUCLEI  in order to understand interactome 

To measure shape, position, size, and interaction of eight organelles/cellular components Nuclei (NU).  

Dependencies:
SOMA and CYTOSOL inference rely on the Nuclei inference.  Therefore all of the sub-cellular objects rely on the NU segmentation.





# IMPORTS

In [2]:

# this needs to be organzied to explain the imports
from pathlib import Path
import os

import numpy as np

import scipy
from scipy import ndimage as ndi

import napari

# function for core algorithm
import aicssegmentation

from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning
from aicssegmentation.core.MO_threshold import MO

from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice

from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage import morphology

from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
# from aicsimageio import AICSImage

from napari.utils.notebook_display import nbscreenshot


%load_ext autoreload
%autoreload 2

if sys.platform == "win32":
    sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))
    from infer_subc.base import *
else:
    from infer_subc.base import *

# OBJECTIVE :  infer NUCLEI
 

NOTE:  using Allen Cell Segmenter  [Nucleophosmin](https://www.allencell.org/cell-observations/category/nucleophosmin) might be a good generic mechanism.  e.g.
-  [playground_npm1.ipynb](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/lookup_table_demo/playground_npm1.ipynb) and [npm1.py](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/aicssegmentation/structure_wrapper/seg_npm1.py) and [npm1_SR.py](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/aicssegmentation/structure_wrapper/seg_npm1_SR.py) 


## summary of steps


INPUT
- channel 0 from raw data :: raw_nuclei

PRE-PROCESSING
-  scale to min 0, max 1.0
- median Filter window 4

CORE-PROCESSING
  - threshold method minimum cross-entropy.  (Li's method)
    - objects size: 50-400 pixels, 
    - threshold smoothing scale: 1 pixel
    - threshold correction factor: 1.2 (more stringent)
    - lower / upper bounds  (.1,1)
    - log transformed thresholding
    - fill holes


POST-PROCESSING
  - fill holes
  - remove small objects


OUTPUT
- inferred of NUCLEI :: inferer_NU an image of labels export as ome.tiff


> #### Note:  this initial inferred object -- the Nuclei of the brightest cell -- will be used in inferring the Soma and Cytosol objects.   This is a straightforward procedure, but also note that any inconsistencies will flow into the Soma and Cytosol objects which in turn affect ALL inferred objects.




In [3]:
# assumptions:   .czi "unmixed" collections.
#czi_img_folder = f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data"

data_path = Path( f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data")
czi_img_folder = data_path/"raw"

list_img_files = lambda img_folder,f_type: [os.path.join(img_folder,f_name) for f_name in os.listdir(img_folder) if f_name.endswith(f_type)]

img_file_list = list_img_files(czi_img_folder,'.czi')
print(img_file_list[5])
#Read the data into memeory from the `.czi` files.  (Note: there is also the 2D slice .tif file read for later comparision).  WE will also collect metatdata.


/Users/ahenrie/Projects/Imaging/mcz_subcell/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.czi


In [4]:

test_img_name = img_file_list[5]

img_data, meta_dict = read_input_image(test_img_name)

raw_meta_data, ome_types = get_raw_meta_data(meta_dict)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']



/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [5]:

viewer = napari.view_image(
    img_data,
    channel_axis=0,
    name=channel_names,
    scale=scale
)
viewer.scale_bar.visible = True



# IMAGE PROCESSING Objective 1:  infer NUCLEI
 
 [OUTLINE: Objective #1](#summary-of-objectives)
## summary of steps


INPUT
- channel 0


PRE-PROCESSING
-  scale to min 0, max 1.0
- median Filter window 4
-  gaussian 1.34

CORE-PROCESSING
  - threshold method minimum cross-entropy.  
    - objects 50-400 pixels, 
    - threshold smoothing scale: 1.34 (later 1 pixel
    - threshold correction factor: 0.9 (later 1.2 )
    - lower / upper bounds  (.1,1) ?
    - log transformed thresholding
    - fill holes


POST-PROCESSING
  - fill holes
  - remove small objects


OUTPUT
- labels of NUCLEI


> #### Note:  in later steps we will limit each analysis to a single object, but at this stage we have multiple



NOTE:  using Allen Cell Segmenter  [Nucleophosmin](https://www.allencell.org/cell-observations/category/nucleophosmin) might e a good place to start.  [playground_npm1.ipynb](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/lookup_table_demo/playground_npm1.ipynb) and [npm1.py](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/aicssegmentation/structure_wrapper/seg_npm1.py) and [npm1_SR.py](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/aicssegmentation/structure_wrapper/seg_npm1_SR.py) 

In [6]:


chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'NU_object'


In [18]:
##########################################################################
# DEFAULT PARAMETERS:
#   note that these parameters are supposed to be fixed for the structure
#   and work well accross different datasets
from collections import defaultdict
# default_params = defaultdict(str)

default_params = defaultdict(str, **{
    #"intensity_norm_param" : [0.5, 15]
    "intensity_norm_param" : [0],
    "gaussian_smoothing_sigma" : 1.34,
    "gaussian_smoothing_truncate_range" : 3.0,
    "dot_2d_sigma" : 2,
    "dot_2d_sigma_extra" : 1,
    "dot_2d_cutoff" : 0.025,
    "min_area" : 10,
    "low_level_min_size" :  100,
    "median_filter_size" : 4
})


################################

# calculate a filter dimension for median filtering which considers the difference in scale of Z
z_factor = scale[0]//scale[1]
med_filter_size = 4 #2D 
med_filter_size_3D = (1,med_filter_size,med_filter_size)  # set the scale for a typical median filter
print(f"median filtering scale is ~ : { [x*y for x,y in zip(scale,med_filter_size_3D)]}")

default_params['z_factor'] = z_factor
default_params['scale'] = scale


gaussian_smoothing_sigma = 1.34
gaussian_smoothing_truncate_range = 3.0


median filtering scale is ~ : [0.5804527163320905, 0.3194866073934927, 0.3194866073934927]


DATA IMPORT

Get the "raw" signals we need to analyze as well as any other dependencies in "inferred" objects.  

> NOTE: we are operating on a single "test" image in this notebook.  The batch-processing of all the images will be happen at the end of the notebook after we have developed/confirmed the setmentation procedures and parameter settings.

In [19]:


###################
# INPUT
###################
raw_nuclei = img_data[0,:,:,:].copy()
# raw_lyso    = img_data[1,:,:,:].copy()
# raw_mito    = img_data[2,:,:,:].copy()
# raw_golgi   = img_data[3,:,:,:].copy()
# raw_peroxi = img_data[4,:,:,:].copy()
# raw_ER      = img_data[5,:,:,:].copy()
# raw_lipid   = img_data[6,:,:,:].copy()
# raw_residual = img_data[7,:,:,:].copy()
# total_flourescence = intensity_normalization(img_data.copy(), scaling_param=[0]).sum(axis=0)
# total_flourescence_scaled = intensity_normalization(total_flourescence, scaling_param=[0])



## PRE- PROCESSING

In [20]:


###################
# PRE_PROCESSING
###################                         )
struct_img = intensity_normalization(raw_nuclei.copy(), scaling_param=[0])

# structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
# # very little difference in 2D vs 3D
structure_img_median = median_filter_slice_by_slice( 
                                                                struct_img,
                                                                size=med_filter_size  )


structure_img_smooth = image_smoothing_gaussian_slice_by_slice(   structure_img_median,
                                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                                        truncate_range=gaussian_smoothing_truncate_range,
                                                                                                                    )


intensity normalization: min-max normalization with NO absoluteintensity upper bound



> #### NOTE: Thresholding
> [Thresholding](https://en.wikipedia.org/wiki/Thresholding_%28image_processing%29) is used to create binary images. A threshold value determines the intensity value separating foreground pixels from background pixels. Foregound pixels are pixels brighter than the threshold value, background pixels are darker. In many cases, images can be adequately segmented by thresholding followed by labelling of *connected components*, which is a fancy way of saying "groups of pixels that touch each other".
> 
> Different thresholding algorithms produce different results. [Otsu's method](https://en.wikipedia.org/wiki/Otsu%27s_method) and [Li's minimum cross entropy threshold](https://scikit-image.org/docs/dev/auto_examples/developers/plot_threshold_li.html) are two common algorithms. Below, we use Li. You can use `skimage.filters.threshold_<TAB>` to find different thresholding methods.

_Li_ procedure  better matches the CellProfiler pipeline which simply calls it "Minimum Cross Entropy" .


## CORE PROCESSING

In [27]:

###################
# CORE_PROCESSING
###################

#tol = max(numpy.min(numpy.diff(numpy.unique(structure_img_median))) / 2, 0.5 / 65536) #assumes 16bit?
#threshold_value = filters.threshold_li(structure_img_smooth)

threshold_value_log = threshold_li_log(structure_img_smooth)

threshold_factor = 0.9 #from cellProfiler
thresh_min = 0.1
thresh_max = 1.0
threshold = min( max(threshold_value_log*threshold_factor, thresh_min), thresh_max)

threshold

0.1

In [28]:


li_thresholded = structure_img_smooth > threshold


## POST PROCESSING

In [29]:
###################
# POST_PROCESSING
###################

hole_width = 5  
# # wrapper to remoce_small_objects
removed_holes = morphology.remove_small_holes(li_thresholded, hole_width ** 3 )


small_object_max = 5
cleaned_img = aicssegmentation.core.utils.size_filter(removed_holes, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= small_object_max**3, 
                                                         method = "slice_by_slice", #"3D", # 
                                                         connectivity=1)



OUTPUT + Visualize Results

In [30]:
NU_object = cleaned_img
NU_labels = label(cleaned_img   )
NU_signal = struct_img

In [31]:
viewer.add_image(
    NU_object,
    scale=scale,
    opacity=0.3,
)    


viewer.add_labels(
    NU_labels,
    scale=scale,
    opacity=0.3,
)



<Labels layer 'NU_labels [2]' at 0x1749d2eb0>

# DEFINE `infer_NUCLEI` function

In [32]:
# copy this to base.py for easy import

def infer_NUCLEI(struct_img, in_params) -> tuple:
    """
    Procedure to infer NUCLEI from linearly unmixed input.

    Parameters:
    ------------
    struct_img: np.ndarray
        a 3d image containing the NUCLEI signal

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of NU
        label
            label (could be more than 1)
        signal
            scaled/filtered (pre-processed) flourescence image
        parameters: dict
            updated parameters in case any needed were missing
    
    """

    out_p= in_params.copy()


    ###################
    # PRE_PROCESSING
    ###################                         

    #TODO: replace params below with the input params
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param

    med_filter_size = 4   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice( 
                                                                    struct_img,
                                                                    size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 1.34
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    ###################
    # CORE_PROCESSING
    ###################

    struct_obj = struct_img > filters.threshold_li(struct_img)
    threshold_value_log = threshold_li_log(struct_img)

    threshold_factor = 0.9 #from cellProfiler
    thresh_min = .1
    thresh_max = 1.
    threshold = min( max(threshold_value_log*threshold_factor, thresh_min), thresh_max)
    out_p['threshold_factor'] = threshold_factor
    out_p['thresh_min'] = thresh_min
    out_p['thresh_max'] = thresh_max

    struct_obj = struct_img > threshold

    ###################
    # POST_PROCESSING
    ###################

    hole_width = 5  
    # # wrapper to remoce_small_objects
    struct_obj = morphology.remove_small_holes(struct_obj, hole_width ** 3 )
    out_p['hole_width'] = hole_width


    small_object_max = 5
    struct_obj = aicssegmentation.core.utils.size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max**3, 
                                                         method = "slice_by_slice", #"3D", # 
                                                            connectivity=1)
    out_p['small_object_max'] = small_object_max


    retval = (struct_obj,  label(struct_obj), out_p)
    return retval


In [33]:
# test - 9.7 secods to run...

NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei.copy(), default_params) 


intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [34]:
# TODO:  make export ome_tiff export:   XX_object, XX_label, XX_signal
#              also fix Path vs. str action for export wrapper

chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'NU_object'

NU_object_filen = export_ome_tiff(NU_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)

['NU_object']


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [35]:
NU_object_filen

'/Users/ahenrie/Projects/Imaging/mcz_subcell/data/inferred_objects/NU_object.ome.tiff'

In [36]:
out_p

defaultdict(str,
            {'intensity_norm_param': [0],
             'gaussian_smoothing_sigma': 1.34,
             'gaussian_smoothing_truncate_range': 3.0,
             'dot_2d_sigma': 2,
             'dot_2d_sigma_extra': 1,
             'dot_2d_cutoff': 0.025,
             'min_area': 10,
             'low_level_min_size': 100,
             'median_filter_size': 4,
             'z_factor': 7.0,
             'scale': (0.5804527163320905,
              0.07987165184837318,
              0.07987165184837318),
             'threshold_factor': 0.9,
             'thresh_min': 0.1,
             'thresh_max': 1.0,
             'hole_width': 5,
             'small_object_max': 5})